### John Hopkins GitHub Repo Data Wrangling

In [1]:
import json
import os
import sys
import pandas as pd
import numpy as np
import boto3
import uuid
from slugify import slugify

In [2]:
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(__name__))), 'COVID-19')

In [3]:
# the confirmed cases time series files in here have Lat and Long for each location
# confirmed_series_file = os.path.join(
#     DATA_DIR,
#     'csse_covid_19_data',
#     'csse_covid_19_time_series',
#     'time_series_covid19_confirmed_global.csv'
# )
# confirmed_series_file

In [4]:
# show first 10 rows of time_series_19-covid-Confirmed.csv file
#!awk -F, '{print $1,$2,$3,$4} NR==10{exit}' OFS=', ' \
# COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv

In [5]:
# cols=['Province/State', 'Country/Region', 'Lat', 'Long']
# locations_df = pd.read_csv(confirmed_series_file, usecols=cols)
# locations_df.head()

In [6]:
s3_resource = boto3.resource('s3')
bucket_name = 'thecodinginterface-covid'
s3_bucket = s3_resource.Bucket(name=bucket_name)

In [7]:
def slugify_location(country_region, province_state):
    if province_state:
        return slugify(f"{country_region}-{province_state}")
    return slugify(country_region)

def cloud_resource_url(filename, bucket_name):
    return f"https://{bucket_name}.s3.amazonaws.com/{filename}.json"

def upload_file_to_s3(s3_bucket, file_path, file_name):
    s3_bucket.upload_file(
        Filename=file_path,
        Key=file_name,
        ExtraArgs={'ACL':'public-read'}
    )
    return cloud_resource_url(file_name, s3_bucket.name)

In [8]:
# rename columns to be snake_cased making it more ammenable to serialization
# locations_df = locations_df.rename(columns={
#     'Province/State': 'province_state',
#     'Country/Region': 'country_region',
#     'Lat': 'lat',
#     'Long': 'long'
# })

# # make sure text columns are well cleaned and stripped of whitespace
# locations_df.province_state = locations_df.province_state.str.strip()
# locations_df.country_region = locations_df.country_region.str.strip()

# # Fill NaNs with empty strings in the Province/State columns because this data will
# # be serialized into JSON which does not support NaN
# locations_df.province_state = locations_df.province_state.fillna('')

# # create columns "filename" and "cloud_resource"
# lookup_keys = zip(locations_df.country_region, locations_df.province_state)
# locations_df['location_id'] = [slugify_location(country_region, province_state)
#                             for country_region, province_state in lookup_keys]

# locations_df['cloud_resource'] = [cloud_resource_url(filename, bucket_name)
#                                   for filename in locations_df['location_id'].values]

# locations_df.head()

In [9]:
# locations_df.country_region.unique()

In [10]:
# locations_df = locations_df.set_index('location_id')
# locations_df[locations_df.country_region == 'US'].sort_values('province_state')

In [11]:
# I'll do more with this locations_df DataFrame later after
# constructing country specific case data sets

In [12]:
# build list of daily csv files
daily_series_dir = os.path.join(
    DATA_DIR,
    'csse_covid_19_data',
    'csse_covid_19_daily_reports'
)
daily_csv_files = [file_name
                   for file_name in os.listdir(daily_series_dir) 
                   if file_name.endswith('csv')]
daily_csv_files[:5]

['02-26-2020.csv',
 '02-27-2020.csv',
 '04-08-2020.csv',
 '04-09-2020.csv',
 '02-18-2020.csv']

In [13]:
# take a peek at the structure of a file that will be worked with
os.path.join(daily_series_dir, daily_csv_files[-1])

'/Users/adammcquistan/Code/python/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-08-2020.csv'

In [14]:
#!head ./COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/02-26-2020.csv

In [15]:
def calc_differential(x):
    x0 = np.array([0] + x[:-1].tolist())
    dx = x.values - x0
    return dx

In [16]:
# read the daily files into DataFrame objects then concatenate them together
daily_dfs = []
colunns_of_interest = [
    'province_state',
    'country_region',
    'total_confirmed',
    'total_deaths',
    'total_recovered',
    'date'
]

for file_name in daily_csv_files:
    file_path = os.path.join(daily_series_dir, file_name)
    day_df = pd.read_csv(file_path)

    # Province_State and Country_Region replaced column names Province/State
    # and Country/Region for new daily files starting 03-24-2020
    day_df = day_df.rename(columns={
            'Province/State': 'province_state',
            'Province_State': 'province_state', 
            'Country/Region': 'country_region',
            'Country_Region': 'country_region',
            'Confirmed': 'total_confirmed',
            'Deaths':'total_deaths',
            'Recovered': 'total_recovered'
        })
    
    date_str, ext = os.path.splitext(file_name)
    num_rows = day_df.shape[0]
    day_df['date'] = [pd.to_datetime(date_str)] * num_rows

    # Fill NaNs with empty strings because this data will
    # be serialized into JSON which does not support NaN
    day_df.province_state = day_df.province_state.fillna('')
    day_df.total_confirmed = day_df.total_confirmed.fillna(0)
    day_df.total_deaths = day_df.total_deaths.fillna(0)
    day_df.total_recovered = day_df.total_recovered.fillna(0)

    missing_columns = sum([(col not in day_df.columns) for col in colunns_of_interest])
    if missing_columns:
        print('Missing Columns!!!!')
        sys.exit(0)

    whole_country_df = day_df[colunns_of_interest].groupby(['country_region']).sum()
    whole_country_df = whole_country_df.reset_index()
    whole_country_df['province_state'] = [''] * whole_country_df.shape[0]
    whole_country_df['date'] = [day_df.date.values[0]] * whole_country_df.shape[0]
    whole_country_df.loc[whole_country_df['country_region'] == 'US', ['country_region']] = 'United States'
        
    if 'Admin2' in day_df.columns:
        county_df = day_df[day_df.Admin2 != '']
        county_df['province_state'] = county_df.Admin2 + ', ' + county_df.province_state
        day_df = pd.concat([day_df[colunns_of_interest], county_df[colunns_of_interest], whole_country_df])
    else:
        day_df = pd.concat([day_df[colunns_of_interest], whole_country_df])
    
#     day_df = day_df[colunns_of_interest]
    
    # increased granularity by neighborhood was added in Admin2 column 03-24-2020
    # but only want granularity down to province_region so collapse down and aggregate
    day_df = day_df.groupby(['country_region', 'province_state', 'date']).sum()
    day_df = day_df.reset_index()
    
    daily_dfs.append(day_df[colunns_of_interest])
    
daily_df = pd.concat(daily_dfs)
# make sure text columns are well cleaned and stripped of whitespace
daily_df.province_state = daily_df.province_state.str.strip()
daily_df.country_region = daily_df.country_region.str.strip()
daily_df.head(10)

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date
0,,Afghanistan,2.0,0.0,0.0,2020-02-26
1,,Algeria,2.0,0.0,0.0,2020-02-26
2,,Australia,22.0,0.0,11.0,2020-02-26
3,From Diamond Princess,Australia,7.0,0.0,0.0,2020-02-26
4,New South Wales,Australia,4.0,0.0,4.0,2020-02-26
5,Queensland,Australia,5.0,0.0,1.0,2020-02-26
6,South Australia,Australia,2.0,0.0,2.0,2020-02-26
7,Victoria,Australia,4.0,0.0,4.0,2020-02-26
8,,Austria,4.0,0.0,0.0,2020-02-26
9,,Bahrain,66.0,0.0,0.0,2020-02-26


In [17]:
daily_df[daily_df['country_region'] == 'United States'].sort_values('date')

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date
42,,United States,1.0,0.0,0.0,2020-01-22
49,,United States,1.0,0.0,0.0,2020-01-23
44,,United States,2.0,0.0,0.0,2020-01-24
47,,United States,2.0,0.0,0.0,2020-01-25
51,,United States,5.0,0.0,0.0,2020-01-26
...,...,...,...,...,...,...
2810,,United States,337072.0,9619.0,17448.0,2020-04-05
2855,,United States,366667.0,10783.0,19581.0,2020-04-06
2903,,United States,396223.0,12722.0,21763.0,2020-04-07
2929,,United States,429052.0,14695.0,23559.0,2020-04-08


In [18]:
# whole_country_df = daily_df.groupby(['country_region', 'date']).sum()
# whole_country_df = whole_country_df.reset_index()
# whole_country_df['province_state'] = [''] * whole_country_df.shape[0]
# whole_country_df.head(10)

In [19]:
# daily_df = pd.concat([daily_df, whole_country_df])
daily_df = daily_df.drop_duplicates()
locations = zip(daily_df.country_region.values, daily_df.province_state.values)
daily_df['location_id'] = [slugify_location(country_region, province_state)
                           for country_region, province_state in locations]

# sort by country_region, province_state, date
daily_df = daily_df.sort_values(['country_region', 'province_state', 'date'])
daily_df[(daily_df.country_region == 'US') & (daily_df.province_state == 'New York')]

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date,location_id
190,New York,US,173.0,0.0,0.0,2020-03-10,us-new-york
197,New York,US,220.0,0.0,0.0,2020-03-11,us-new-york
199,New York,US,328.0,0.0,0.0,2020-03-12,us-new-york
211,New York,US,421.0,0.0,0.0,2020-03-13,us-new-york
225,New York,US,525.0,2.0,0.0,2020-03-14,us-new-york
233,New York,US,732.0,3.0,0.0,2020-03-15,us-new-york
247,New York,US,967.0,10.0,0.0,2020-03-16,us-new-york
251,New York,US,1706.0,13.0,0.0,2020-03-17,us-new-york
256,New York,US,2495.0,16.0,0.0,2020-03-18,us-new-york
263,New York,US,5365.0,34.0,0.0,2020-03-19,us-new-york


In [20]:
entry_counts = daily_df.groupby('location_id').count().sort_values('date')
entry_counts = entry_counts[entry_counts.date < 20]
for loc_id in entry_counts.index.values:
    daily_df = daily_df[daily_df.location_id != loc_id]

In [21]:
# get totals per location
max_date = daily_df.date.max()
rows_of_interest = daily_df.date == max_date
columns_of_interest = [
    'location_id',
    'total_confirmed',
    'total_deaths',
    'total_recovered'
]
location_totals_df = daily_df.loc[rows_of_interest, columns_of_interest]
location_totals_df = location_totals_df.groupby('location_id').sum()
location_totals_df['death_rate'] = location_totals_df.total_deaths / location_totals_df.total_confirmed * 100
location_totals_df['recovery_rate'] = location_totals_df.total_recovered / location_totals_df.total_confirmed * 100
location_totals_df.sort_values('total_confirmed', ascending=False).head(10)

,total_confirmed,total_deaths,total_recovered,death_rate,recovery_rate
location_id,,,,,
united-states,461437.0,16478.0,25410.0,3.571018,5.506711
spain,306444.0,30894.0,104330.0,10.081450,34.045372
italy,287252.0,36558.0,56940.0,12.726804,19.822316
france,236530.0,24438.0,46619.0,10.331882,19.709551
germany,236362.0,5214.0,104814.0,2.205938,44.344692
us-new-york,161779.0,7067.0,0.0,4.368305,0.000000
iran,132440.0,8220.0,64618.0,6.206584,48.790396
united-kingdom,130949.0,15971.0,494.0,12.196351,0.377246
us-new-york-city-new-york,87028.0,5150.0,0.0,5.917636,0.000000


In [22]:
# get totals per country / region
# country_totals_df = locations_df.join(location_totals_df)
# columns_of_interest = [
#     'country_region',
#     'total_confirmed',
#     'total_deaths',
#     'total_recovered',
# ]
# country_totals_df = country_totals_df.reset_index()
# country_totals_df = country_totals_df[columns_of_interest].groupby('country_region').sum()
# country_totals_df['death_rate'] = country_totals_df.total_deaths / country_totals_df.total_confirmed * 100
# country_totals_df['recovery_rate'] = country_totals_df.total_recovered / country_totals_df.total_confirmed * 100
# country_totals_df.sort_values('country_region').head(25)

In [23]:
# world_population_df = pd.read_csv('world_population.csv')
# world_population_df = world_population_df.set_index('country_region')
# world_population_df.head()

In [24]:
# # add population data to country totals
# country_totals_df = country_totals_df.join(world_population_df)
# country_totals_df.head(20)

In [25]:
daily_df[(daily_df.country_region == 'US') & (daily_df.location_id == 'us-new-york-city-new-york')]

,province_state,country_region,total_confirmed,total_deaths,total_recovered,date,location_id
2334,"New York City, New York",US,9654.0,63.0,0.0,2020-03-22,us-new-york-city-new-york
2329,"New York City, New York",US,12305.0,99.0,0.0,2020-03-23,us-new-york-city-new-york
2330,"New York City, New York",US,14904.0,131.0,0.0,2020-03-24,us-new-york-city-new-york
2334,"New York City, New York",US,17856.0,199.0,0.0,2020-03-25,us-new-york-city-new-york
2336,"New York City, New York",US,21873.0,281.0,0.0,2020-03-26,us-new-york-city-new-york
2337,"New York City, New York",US,25573.0,366.0,0.0,2020-03-27,us-new-york-city-new-york
2338,"New York City, New York",US,29776.0,517.0,0.0,2020-03-28,us-new-york-city-new-york
2338,"New York City, New York",US,33768.0,678.0,0.0,2020-03-29,us-new-york-city-new-york
2339,"New York City, New York",US,37453.0,790.0,0.0,2020-03-30,us-new-york-city-new-york
1671,"New York City, New York",US,43119.0,932.0,0.0,2020-03-31,us-new-york-city-new-york


In [26]:
# group by location and serialize each location dataset to a json file
# [ 
#   {
#     date: str,
#     province_state: str,
#     confirmed: int,
#     deaths: int,
#     recovered: int
#   }, ...
# ]

location_case_data = 'location_case_data'
if not os.path.exists(location_case_data):
    os.mkdir(location_case_data)

location_groups = daily_df.groupby(['location_id'])
groups = len(location_groups)
i = 1
for location_id, location_data in location_groups:
    location_data = location_data.sort_values('date')
    location_data.loc[:,'daily_confirmed'] = calc_differential(location_data.total_confirmed)
    location_data.loc[:,'daily_deaths'] = calc_differential(location_data.total_deaths)
    location_data.loc[:,'daily_recovered'] = calc_differential(location_data.total_recovered)
    
    location_days = []

    for idx, row in location_data.iterrows():
        data = row.to_dict()
        # dates don't serialize well in Python so, convert to strings
        data['date'] = data['date'].strftime('%Y-%m-%d')
        location_days.append(data)

    filename = f"{location_id}.json"
    file_path = os.path.join(location_case_data, filename)

    with open(file_path, 'w') as fo:
        json.dump(location_days, fo, indent=4)
    
    if i % 100 == 0:
        print(f"Uploading {location_id} {i} of {groups}")
    i += 1

    s3_url = upload_file_to_s3(s3_bucket, file_path, filename)


Uploading azerbaijan 20 of 2948
Uploading canada 40 of 2948
Uploading chile 60 of 2948
Uploading china-jilin 80 of 2948
Uploading croatia 100 of 2948
Uploading ethiopia 120 of 2948
Uploading grenada 140 of 2948
Uploading japan 160 of 2948
Uploading mainland-china-anhui 180 of 2948
Uploading mainland-china-qinghai 200 of 2948
Uploading monaco 220 of 2948
Uploading others-diamond-princess-cruise-ship 240 of 2948
Uploading seychelles 260 of 2948
Uploading trinidad-and-tobago 280 of 2948
Uploading us-acadia-louisiana 300 of 2948
Uploading us-alachua-florida 320 of 2948
Uploading us-allen-louisiana 340 of 2948
Uploading us-antelope-nebraska 360 of 2948
Uploading us-ashley-arkansas 380 of 2948
Uploading us-bacon-georgia 400 of 2948
Uploading us-barry-missouri 420 of 2948
Uploading us-bedford-pennsylvania 440 of 2948
Uploading us-berkeley-south-carolina 460 of 2948
Uploading us-blanco-texas 480 of 2948
Uploading us-boston-ma 500 of 2948
Uploading us-breckinridge-kentucky 520 of 2948
Uploading

In [27]:
# locations_df.head(25)
# locations_df = locations_df.reset_index()
# locations_df.head()

In [28]:
locations_df = daily_df[['location_id', 'province_state', 'country_region']].drop_duplicates()
locations_df['cloud_resource'] = [cloud_resource_url(location_id, bucket_name) 
                                  for location_id in locations_df.location_id.values]
locations_df.head()

,location_id,province_state,country_region,cloud_resource
0,afghanistan,,Afghanistan,https://thecodinginterface-covid.s3.amazonaws....
1,albania,,Albania,https://thecodinginterface-covid.s3.amazonaws....
1,algeria,,Algeria,https://thecodinginterface-covid.s3.amazonaws....
2,andorra,,Andorra,https://thecodinginterface-covid.s3.amazonaws....
4,angola,,Angola,https://thecodinginterface-covid.s3.amazonaws....


In [29]:
locations_df[locations_df.province_state == 'Nebraska']

,location_id,province_state,country_region,cloud_resource
185,us-nebraska,Nebraska,US,https://thecodinginterface-covid.s3.amazonaws....


In [ ]:
# create a list of dicts in the form:
# [ 
#   {
#     country_region: str,
#     province_state: str,
#     lat: float,
#     long: float,
#     filename: str,
#     cloud_resource: str
#   },
#    ...
# ]
locations = []
location_groups = locations_df.groupby(['location_id'])
for k, location_data in location_groups:
    for i, row in location_data.iterrows():
        data = row.to_dict()
        locations.append(data)
        if i < 5:
            print(data)

In [26]:
# serialize locations to JSON file
with open('locations.json', 'w') as fo:
    json.dump(locations, fo, indent=4)

#!head -n 15 locations.json

## Country / Region Dashboard

Give user ability to select (aka drill down) into country, region, state, province

Show confirmed, deaths, recovered

Show time series of total confirmed

Show time series of total recovered

Show time series of total deaths

Show time series of daily new confirmed

Show time series of daily new recovered

Show time series of daily new deaths

Would be interesting to give a Gauge chart next to the daily graphs with an indicator of direction of n day movement (ie, over the last three days is new daily cases (deaths, confirmed, recovered) increasing, descreasing, maintaining)


## Location Comparisons

### Barcharts

Death Rates: select locations to include and date in time (includes checkbox to make percent of population)

Confirmed Counts: select locations to include and date in time (includes checkbox to make percent of population)


### Line Charts

Total Confirmed Cases: select locations to include and plot progression of cases since first case (includes checkbox to make percent of population)

New Daily Confirmed Cases: select locations to include and plot progression of new cases since first case in each location (includes checkbox to make percent of population)

Total Deaths: select locations to include and plot progression of deaths since first case (includes checkbox to make percent of population)

New Daily Deaths: select locations to include and plot progression of deaths since first  case in each location (includes checkbox to make percent of population)

In [23]:
locations_df.to_csv('locations.csv')